# Travaux Pratiques d'imagerie par rayons X
NB : Vous pouvez travailler en groupe mais le travail est à rendre individuellement.

In [1]:
import numpy as np
import xraylib as xrl
from xpecgen import xpecgen as xg
import ipywidgets as widgets
from tabletext import to_text
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.ticker as tic
import matplotlib as mpl
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter, AutoMinorLocator)

# Loi d'atténuation
Le but est de retrouver le coefficient d’atténuation à 100 keV de matériaux à travers une simulation réaliste en utilisant la loi de Beer-Lambert :
$$
N_{\mathrm{dt}}=N_0\exp(-\mu(E)X)
$$
où $N_0$ est le nombre de photons incident sur l'objet d'épaisseur $X$ et $\mu$ le coefficient d'atténuation linéique à l'énergie $E$.

## Conditions expérimentales
Le code suivant simule un nombre de photons détectés après avoir traversé 0.1 mm de plomb (Pb, densité 11.35 g.cm$^{⁻3}$) et 3.9 cm de polyéthylène (H$_2$C, densité 0.94 g.cm$^{⁻3}$) étant donné un nombre de photons incidents. Le processus stochastique d'atténuation est reproduit (par une loi de probabilité de Poisson).

## Travail demandé
Retrouvez le coefficient d'atténuation du plomb et du polyéthylène à 100 keV avec une précision de $10^{-3}$. Il vous faudra adapter le nombre de photons envoyés. Pour mémoire, la loi de propagation d'erreur d'une fonction $y(x_1, ..., x_N)$ est
$$
\mathrm{Var}[y]=\sum_{i=1}^N\left(\dfrac{\partial y}{\partial x_i}\right)^2\mathrm{Var}[x_i]
$$
où la variance $\mathrm{Var}[]$ représente le carré de l'erreur (ou écart-type).

Vérifiez la concordance du résultat obtenu avec le coefficient d’atténuation linéique théorique obtenu sur la base de données [XCOM](http://physics.nist.gov/PhysRefData/Xcom/html/xcom1.html) de [NIST](http://www.nist.gov/pml/data/xray_gammaray.cfm).

In [2]:
def formula(compound):
    elt_CP = [xrl.AtomicNumberToSymbol(compound['Elements'][i]) for i in range(compound['nElements'])]
    atf_CP = [compound['massFractions'][i]/xrl.AtomicWeight(compound['Elements'][i])
              for i in range(compound['nElements'])]
    atfn_CP = [str(atf/min(atf_CP)) for atf in atf_CP]
    return "".join(np.core.defchararray.add(elt_CP,atfn_CP))

def simulator(N0):
    cpH2C = xrl.GetCompoundDataNISTByName('Polyethylene') 
    dH2C = cpH2C['density']
    fPE = formula(cpH2C)
    NdtH2C = np.random.poisson(N0*np.exp(-3.9*dH2C*xrl.CS_Total_CP(fPE, 100)))
    print('%d photons ont traversé la plaque de 3.9 cm de polyéthylène sur %d envoyés' % (NdtH2C,N0))
    dPb = xrl.ElementDensity(82)
    NdtPb = np.random.poisson(N0*np.exp(-0.01*dPb*xrl.CS_Total(82, 100)))
    print('%d photons ont traversé la plaque de 0.1 mm de plomb sur %d envoyés' % (NdtPb,N0))

widgets.interact_manual(simulator,
                        N0=widgets.IntText(value=100,description="Nombre de photons émis :",disabled=False,
                                           style={'description_width': 'initial'}));

interactive(children=(IntText(value=100, description='Nombre de photons émis :', style=DescriptionStyle(descri…

# Étude du polychromatisme

La largeur du spectre en énergie des photons incidents a une forte influence sur la formation de
l’image radiologique. Nous l’étudions ici.

## Conditions expérimentales. 

Le logiciel de simulation ci-dessous permet de calculer des spectres tels que générés par les générateurs de rayons X en précisant :
- la haute tension U d’accélération des électrons
- l'épaisseur et la nature d'une plaque insérée dans le faisceau 

Certaines grandeurs associées aux spectres sont calculées en sortie notamment
- la couche de demi-atténuation (en mm d’Al),
- l’énergie moyenne des photons du spectre

Nota Bene : caractéristiques du tube X
- réglé pour avoir 1mGy à 1m dans l'air en absence objet
- filtration interne de 1.2mm d'aluminium


## Travail demandé. 

Nous allons observer l’évolution du spectre en fonction de l’épaisseur et de la nature d'une plaque insérée dans le faisceau. Utilisez les widgets pour sélectionner le matériau et l'épaisseur de la plaque.
- Tracez l'énergie moyenne des photons transmis derrière une plaque en fonction de son épaisseur pour des plaques en cuivre puis en plomb. Quelles sont les valeurs limites ?
- Expliquez la différence entre les deux spectres transmis (plomb vs cuivre) en regardant les coefficients d'atténuation correspondants, disponibles dans le jupyter notebook ci-dessous.
- Déterminez pour ce spectre de rayons X de 100 kV les 10 premiers HVL (couche de demi-valeur) exprimés en mm d'aluminium. Expliquez la valeur limite.
- Pour l'ensemble des matériaux proposés dans le menu déroulant, trouvez l'épaisseur de plaque correspondante qui supprime les énergies en dessous de 40 keV. Expliquez les différences. Vérifiez que ce filtre fonctionne également pour d'autres tensions.

In [3]:
def spectrum(E0,Mat_Z,Mat_X):
    xrs=xg.calculate_spectrum(E0,12,3,100,epsrel=0.5,monitor=None,z=74)
    #Inherent filtration: 1.2mm Al + 100cm Air
    mu_Al=xg.get_mu(13)
    xrs.attenuate(0.12,mu_Al)
    xrs.attenuate(100,xg.get_mu("air"))
    fluence_to_dose=xg.get_fluence_to_dose()
    xrs.set_norm(value=0.146,weight=fluence_to_dose)
    #Attenuation
    if Mat_Z>0: #Atomic number
        dMat = xrl.ElementDensity(Mat_Z)
        fMat = xrl.AtomicNumberToSymbol(Mat_Z)
        xrs.attenuate(0.1*Mat_X,xg.get_mu(Mat_Z))
    else: #-1 == 'Water'
        mH2O = 2. * xrl.AtomicWeight(1) + xrl.AtomicWeight(8)
        wH = 0.1 * Mat_X * 2. * xrl.AtomicWeight(1) / (xrl.ElementDensity(1) * mH2O)
        wO = 0.1 * Mat_X * xrl.AtomicWeight(8) / (xrl.ElementDensity(8) * mH2O)
        xrs.attenuate(wH,xg.get_mu(1))
        xrs.attenuate(wO,xg.get_mu(8))
    #Get the figures
    Nr_Photons = "%.4g" % (xrs.get_norm())
    Average_Energy = "%.2f keV" % (xrs.get_norm(lambda x:x)/xrs.get_norm())
    Dose = "%.3g mGy" % (xrs.get_norm(fluence_to_dose))    
    HVL_Al=xrs.hvl(0.5,fluence_to_dose,mu_Al)
    HVL_Al_text = "%.2f mm (Al)" % (10*HVL_Al)
    a = [["Dose à 1m", Dose],["Nombre total de photons", Nr_Photons],
         ["Énergie moyenne des photons",Average_Energy],["Couche de Demie-Atténuation", HVL_Al_text]]
    print(to_text(a))
    (x2,y2) = xrs.get_points()
    plt.figure(1,dpi=150)
    mpl.rcParams.update({'font.size': 6})
    axMW = plt.subplot(111)
    axMW.plot(x2,y2)
    axMW.set_xlim(3,E0)
    axMW.set_ylim(0,)
    plt.xlabel("Énergie [keV]")
    plt.ylabel("Nombre de photons par [keV·cm²·mGy] @ 1m")
    axMW.grid(which='major', axis='x', linewidth=0.5, linestyle='-', color='0.75')
    axMW.grid(which='minor', axis='x', linewidth=0.2, linestyle='-', color='0.85')
    axMW.grid(which='major', axis='y', linewidth=0.5, linestyle='-', color='0.75')
    axMW.grid(which='minor', axis='y', linewidth=0.2, linestyle='-', color='0.85')
    axMW.xaxis.set_major_formatter(tic.FormatStrFormatter("%d"))
    axMW.yaxis.set_major_formatter(tic.FormatStrFormatter("%.2g"))
    axMW.xaxis.set_minor_locator(AutoMinorLocator())
    axMW.yaxis.set_minor_locator(AutoMinorLocator())
    axMW.grid(True)
    plt.show()

widgets.interact_manual(spectrum, 
                E0 = widgets.IntText(value=100,step=1,description="Haute tension (kV) :",disabled=False,
                                     style={'description_width': 'initial'}),
                Mat_Z = widgets.Dropdown(options=[('eau',-1),('aluminium',13),('cuivre',29),('plomb',82)],
                                         value=-1,layout={'width': 'max-content'},description='Matériau :',
                                         style={'description_width': 'initial'}),
                Mat_X = widgets.FloatText(value=0.,step=0.01,description="Épaisseur (mm) :",disabled=False,
                                          style={'description_width': 'initial'}));


interactive(children=(IntText(value=100, description='Haute tension (kV) :', style=DescriptionStyle(descriptio…

# Coefficient d'atténuation linéique
$$
\mu = \sum_{i\in\mathsf{Interactions}} \mu_i = \rho \frac{\mathcal{N}_A}{A} \left(
\sum_{i\in\mathsf{Interactions}}\sigma_i\right)
$$
où $\sigma_i$, qui ne dépend pas de la densité $\rho$, est la section efficace (en cm$^2$) de l'interaction $i$ pour le materiau

In [4]:
def mu(Z=13,components=False):
    energy_range = np.arange(5.,800., 0.1, dtype=np.double)
    density = xrl.ElementDensity(Z)
    mu_rho = [xrl.CS_Total(Z, E) * density for E in energy_range]
    if components:
        mu_rho_Photo = [xrl.CS_Photo(Z, E) * density for E in energy_range]
        mu_rho_Compt = [xrl.CS_Compt(Z, E) * density for E in energy_range]
        mu_rho_Rayl = [xrl.CS_Rayl(Z, E) * density for E in energy_range]
    plt.figure(dpi=150)
    mpl.rcParams.update({'font.size': 6})
    axMW = plt.subplot(111)
    axMW.plot(energy_range, mu_rho,color="black",linewidth=2.,linestyle="-",label='Total')
    if components:
        axMW.plot(energy_range, mu_rho_Photo,color="red",linewidth=2.,linestyle="-",label='Photoelectric')
        axMW.plot(energy_range, mu_rho_Compt,color="blue",linewidth=2.,linestyle="-",label='Compton')
        axMW.plot(energy_range, mu_rho_Rayl,color="green",linewidth=2.,linestyle="-",label='Rayleigh')
    axMW.set_xscale('log')
    axMW.set_yscale('log')
    axMW.set_xlim(5.,800.)
    plt.legend(loc='center right', frameon=True)
    plt.xlabel('Énergie (keV)')
    plt.ylabel("Coefficient d'atténuation linéique (cm$^{-1}$)")
    axMW.grid(which='major', axis='x', linewidth=0.5, linestyle='-', color='0.75')
    axMW.grid(which='minor', axis='x', linewidth=0.3, linestyle='-', color='0.75')
    axMW.grid(which='major', axis='y', linewidth=0.5, linestyle='-', color='0.75')
    axMW.grid(which='minor', axis='y', linewidth=0.3, linestyle='-', color='0.75')
    axMW.xaxis.set_major_formatter(tic.FormatStrFormatter("%d"))
    #axMW.xaxis.set_minor_formatter(tic.FormatStrFormatter("%d"))
    axMW.grid(True)
    symbol=xrl.AtomicNumberToSymbol(Z)
    axMW.set_title("%s" % symbol, va='bottom')
    #plt.savefig('mu_over_rho_W.pdf', format='PDF')
    
    plt.show()

widgets.interact(mu,Z=(1,92,1),components=False,continuous_update=False);

interactive(children=(IntSlider(value=13, description='Z', max=92, min=1), Checkbox(value=False, description='…